In [1]:
from langchain.llms import CTransformers
# from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.agents.agent_toolkits import GmailToolkit
from langchain.tools.gmail.utils import build_resource_service, get_gmail_credentials
from langchain.agents import initialize_agent, AgentType
from langchain.chains.summarize import load_summarize_chain

from langchain.chat_models import ChatOpenAI



if __name__ == '__main__':
    llama_llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin', callbacks=[StreamingStdOutCallbackHandler()])
    chat_gpt_llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k')


    template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
Your answer are brief and usually not more thatn 10 words long.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
{text}[/INST]"""

    toolkit = GmailToolkit()

    # Can review scopes here https://developers.google.com/gmail/api/auth/scopes
    # For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
    credentials = get_gmail_credentials(
        token_file="token.json",
        scopes=["https://mail.google.com/"],
        client_secrets_file="credentials.json",
    )
    api_resource = build_resource_service(credentials=credentials)
    toolkit = GmailToolkit(api_resource=api_resource)

    prompt = PromptTemplate(template=template, input_variables=["text"])

    # llm_chain = LLMChain(prompt=prompt, llm=llm)
    # # text = input("Write question:")
    text = "What is my conversation in my gmail with Lucy.Chan@fragomen.com is about?"
    # response = llm_chain.run(text)
    agent = initialize_agent(
        tools=toolkit.get_tools(),
        llm=chat_gpt_llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    print(dir(agent))
    # chain = load_summarize_chain(llm, chain_type="refine")
    # help(chain)
    # agent.run(text)
    #     prompt.format(text=text)
    # )


C:\Users\ILDAR\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


['Config', '__abstractmethods__', '__annotations__', '__call__', '__class__', '__class_getitem__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__ror__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '__weakref__', '_abatch_with_config', '_abc_impl'

In [82]:
agent.agent.llm_chain.prompt.messages[0].prompt.template

'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\ncreate_gmail_draft: Use this tool to create a draft email with the provided message fields., args: {{{{\'message\': {{{{\'title\': \'Message\', \'description\': \'The message to include in the draft.\', \'type\': \'string\'}}}}, \'to\': {{{{\'title\': \'To\', \'description\': \'The list of recipients.\', \'type\': \'array\', \'items\': {{{{\'type\': \'string\'}}}}}}}}, \'subject\': {{{{\'title\': \'Subject\', \'description\': \'The subject of the message.\', \'type\': \'string\'}}}}, \'cc\': {{{{\'title\': \'Cc\', \'description\': \'The list of CC recipients.\', \'type\': \'array\', \'items\': {{{{\'type\': \'string\'}}}}}}}}, \'bcc\': {{{{\'title\': \'Bcc\', \'description\': \'The list of BCC recipients.\', \'type\': \'array\', \'items\': {{{{\'type\': \'string\'}}}}}}}}}}}}\nsend_gmail_message: Use this tool to send email messages. The input is the message, recipients, args: {{{{

In [87]:
# Reduce
reduce_template = """Summarize this LLM prompt with tools to fit to 768 context size. Keep tools by Large Lnaguage Model with smaller context size.
{agent_template}
Summarized Context:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_prompt

PromptTemplate(input_variables=['agent_template'], output_parser=None, partial_variables={}, template='Summarize this LLM prompt with tools to fit to 768 context size. Keep tools by Large Lnaguage Model with smaller context size.\n{agent_template}\nSummarized Context:', template_format='f-string', validate_template=True)

In [88]:
# Run chain
# reduce_chain = LLMChain(llm=chat_gpt_llm, prompt=reduce_prompt)
reduce_chain.run(agent_template = agent.agent.llm_chain.prompt.messages[0].prompt.template)

'The LLM prompt is asking for a summary of a given LLM (Master of Laws) prompt. The summary should be concise and fit within a context size of 768 characters. The prompt also provides access to several tools that can be used to create a draft email, send email messages, search for email messages or threads, fetch an email by message ID, and search for email messages by thread ID. These tools can be used to assist in answering the prompt.'